In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [2]:
from datasets import load_dataset, Audio

minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:20]")

In [3]:
minds = minds.train_test_split(test_size=0.2)
minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [4]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [5]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602baa0fbb1e6d0fbce9214f.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602baa0fbb1e6d0fbce9214f.wav',
  'array': array([-3.8957143e-05, -2.0281275e-04, -2.0516709e-04, ...,
         -1.2579626e-02, -1.1464247e-02, -6.2970519e-03], dtype=float32),
  'sampling_rate': 16000},
 'transcription': 'I need to find out if I probably set up a joint account'}

In [6]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcription"])
    batch["input_length"] = len(batch["input_values"])
    return batch

In [7]:
encoded_minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

#0:   0%|          | 0/4 [00:00<?, ?ex/s]

#1:   0%|          | 0/4 [00:00<?, ?ex/s]

#3:   0%|          | 0/4 [00:00<?, ?ex/s]

#2:   0%|          | 0/4 [00:00<?, ?ex/s]

Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.


#0:   0%|          | 0/1 [00:00<?, ?ex/s]

#1:   0%|          | 0/1 [00:00<?, ?ex/s]

#2:   0%|          | 0/1 [00:00<?, ?ex/s]

#3:   0%|          | 0/1 [00:00<?, ?ex/s]

Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.
Keyword arguments {'sampling_rate': 16000} not recognized.


In [8]:
encoded_minds["train"][0].keys()

dict_keys(['input_values', 'labels', 'input_length'])

In [11]:
import evaluate

wer_metric = evaluate.load("wer")

In [12]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
 
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
 
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
 
    return {"wer": wer}

In [13]:
from transformers import AutoModelForCTC, TrainingArguments, Trainer

model = AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'quantizer.codevectors', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expe

In [14]:
@dataclass
class DataCollatorCTCWithPadding:

    processor: AutoProcessor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        print([{"input_values": feature["input_values"][:10]} for feature in features])

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        labels_batch = self.processor.tokenizer.pad(labels=label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [15]:
training_args = TrainingArguments(
    output_dir="my_awesome_asr_mind_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    group_by_length=True,
    per_device_train_batch_size=8,
    gradient_checkpointing=True,
    per_device_eval_batch_size=8,
    fp16=True,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10
  Number of trainable parameters = 94396320
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing

ValueError: ignored